### Operation scheme and notes

<img src='hardware_scheme.png' width=600/>

**ADclck** = Sampling rate  
#### channels to read:  
```
  channels[0] = 0; //Uref
  channels[1] = 1; //Umod
  channels[2] = 4; //Utpl
  channels[3] = 5; //Uhtr (R)
  channels[4] = 3; //Uaux
//  channels[5] = 5; //Uabs
```
#### gain and flag for each channel to read:  
```
gainlist[7]={1.0,2.0,5.0,10.0,20.0,50.0,100.0}; //the numbers are not like in GUI!!  
...
     gains[iw] = DgainDbd3kX1; //gain of X1
     flags[iw] = DafAnalog|DafBipolar|DafSettle5us; //select Bipolar mode  !!!!!!!!!!!
```

---

### Fast heating mode

#### Make button clicked:  

```void __fastcall TForm1::BitBtn10Click(TObject *Sender)```

reads all cells and generates temperature profile into ```AWbufTref ```
copy it to ```AWbufT```  

converts it to voltage: ```AWbuf[i] = TempToVoltage(AWbufTref[i])```

#### Arm button clicked  
Puts interface into "waiting" mode  
Changes offset to 0.1 if it is 0 and stores it into ```Goff```  
```counts = buffsize```  
Opens graph panel  
```pulseshoot(u0, n1, u1, n2, u2, counts)```


#### Start/Stop button clicked
```fastheatrun()```

#### Fastheatrun function  
Applies 4.5 volts on trigger channel  
```VoltGen(2,4.5)```  
  
...  
Launches acquisition with respect to DAC signal in trigger mode - different for python library  
...

stores data:  
```overload = UnpackData();```

Uref is the applied voltage, that had been sent to the 1st channel:  
```for(i=0;i<buffsize;i++) PulseRef[i] = ADCdata4disp[0][i];```  
```resultdata[i].Uhtr = PulseRef[i+10]; //ref signal```  

Packing the resultsdata:  
```resultdata[i].time = i*1000./ADclck;   //time ms```  
```resultdata[i].amplitude = ADCdata[2][i+10]+Taux; //amplitude lr```  
```resultdata[i].phase = 0.;```  
```resultdata[i].Uhtr = PulseRef[i+10]; //ref signal```  
```resultdata[i].Ttpl = ADCdata[1][i+10]; //amplitude hr, overloaded```  
```Ih = ADCdata[0][i+10]*ihtrk+ihtro;```  
```if (Ih!=0.) Rhtr = (ADCdata[3][i+10]-ADCdata[0][i+10]*1000.+uhtro)*uhtrk / Ih;```  
```resultdata[i].Thtr = thtrk1 + thtrk2*(Rhtr+Rhcorr) + thtrk3*(Rhtr+Rhcorr)*(Rhtr+Rhcorr); // Thtr temp```  
```resultdata[i].Taux = Taux;```  

Saves the data  
**time(ms)**  
```resultdata[i].time = i*1000./ADclck //time ms```  
**ttemp**  
```resultdata[i].amplitude = ADCdata[2][i+10]+Taux; //amplitude lr```  
**ttemp-fit**  
```resultdata[i].phase = 0.;```  
**ttemp-hr**  
```resultdata[i].Ttpl = ADCdata[1][i+10]; //amplitude hr, overloaded```  
**tRef**  
```PulseRef[i] = ADCdata4disp[0][i];``` - voltage, corrected to heatersafevoltage  
```resultdata[i].Uhtr = PulseRef[i+10]; //ref signal```  
**tIhtr**      
0, cannot find in code where it is set to 0, but it i 0 for sure  
**tThtr**  
```Ih = ADCdata[0][i+10]*ihtrk+ihtro;```  
```if (Ih!=0.) Rhtr = (ADCdata[3][i+10]-ADCdata[0][i+10]*1000.+uhtro)*uhtrk / Ih;```  
```resultdata[i].Thtr = thtrk1 + thtrk2*(Rhtr+Rhcorr) + thtrk3*(Rhtr+Rhcorr)*(Rhtr+Rhcorr); // Thtr temp```  
*Rhcorr:*  
```  if (DoRhcorr)
  {
    if (Rhtr!=0.)
    {
      for( iit=0; iit<1000; iit++)
      {
        Thtr = thtrk1 + thtrk2*(Rhtr+Rhcorr) + thtrk3*(Rhtr+Rhcorr)*(Rhtr+Rhcorr);
        iterr = Ttpl+Taux-Thtr;
        if (fabs(iterr)<0.01) break;
        if (fabs(iterr)>10000.)
        {
           Rhcorr=0.;
           break;
        }
        Rhcorr += iterr*0.1;
      }
    }
```
**-----**    
0, cannot find in code where it is set to 0, but it i 0 for sure  
**tTaux**  
```resultdata[i].Taux = Taux;```  

#### Pulseshoot function  
```void pulseshoot(double u0, DWORD n1, double u1, DWORD n2, double u2, DWORD counts)```  
Checks each voltage point with heater safe voltage  
```iampl = (DWORD)(65535 * (AWbuf[i]+10.)/20.);```  
```bufferout2[i]= iampl;```  
Draws the profile on graph panel  
  
Sets mode for channels (0: Uref, 1:Uguard, 2:Trigger)  
```daqDacSetOutputMode(dhandle, DddtLocal, 0, DdomVoltage);
daqDacSetOutputMode(dhandle, DddtLocal, 1, DdomStaticWave);
daqDacSetOutputMode(dhandle, DddtLocal, 2, DdomVoltage);```  

For Uref loads voltage profile from ```bufferout2```:  
```daqDacWaveSetClockSource(dhandle, DddtLocal, 1, DdcsAdcClock);
daqDacWaveSetMode(dhandle, DddtLocal, 1, DdwmNShot, counts);
daqDacWaveSetUserWave(dhandle, DddtLocal, 1);
daqDacWaveSetBuffer(dhandle, DddtLocal, 1, bufferout2, counts, DdtmUserBuffer);
daqDacWaveSetTrig(dhandle, DddtLocal, 1, DdtsImmediate, 0);```

```daqDacWaveArm(dhandle, DddtLocal);```  
  
Set Goff voltage on 0 channel - Utpl!!!  
```ioffset = (DWORD)(65535 * (Goff+10.)/20.);
daqDacWt(dhandle, DddtLocal, 0, ioffset);```  

Sets 0 voltage to the trigger channel? not sure.. valdaq2 is used only in VlotGen function. After clicking 'On' button it is called only once and sets channel 2 to 0  
```daqDacWt(dhandle, DddtLocal, 2, valdaq2);```

#### TempToVoltage function 
```double TempToVoltage(double t)```
Cuts values below zero and above heater safe voltage  
Searches the nearest Voltage value to the input value on the Voltage-Temperature curve. It uses ```VoltageToTemp``` function to calculate some voltage, calculates the difference from input value and devides it by 300. Repeat it again until the difference is less than 0.005.

#### VoltageToTemp function  
```double VoltageToTemp(double v)```  
Converts voltage to temperture Theater(Uheater) with respect to calibration:  
```itk2*v + itk3*v^2 + itk4*v^3```  
Cuts negative values  

#### UnpackData function  
Applies scale and offset for each channel:  
```ADCdata[ich][i] = bufferin[i*chancount+ich]*scale - offset;```  


Unpacks the buffer into ADCdata by channels  
**Some 121 and 11 magic**????   
  
```ADCdata[1][i] = ADCdata[1][i]/121.*1000;  //Umod mV```  
```ADCdata[2][i] = ADCdata[2][i]/11.*1000.;  //Utpl *11 mV```    
```ADCdata[3][i] = ADCdata[3][i]*1000.;      //Uhtr mV```  
  
Than goes a lot of code to calculate average values for each of periods and displaying it in GUI.  
More important: it launches ``` TempRun() ```


#### Notes about calibration
```Rhcorr``` and ```Rhdcorr``` are to be read from calibration file. When button >0< is pressed, the calibration file is being overwritten with new coefficients.  
```  if (DoRhcorr)
  {
    if (Rhtr!=0.)
    {
      for( iit=0; iit<1000; iit++)
      {
        Thtr = thtrk1 + thtrk2*(Rhtr+Rhcorr) + thtrk3*(Rhtr+Rhcorr)*(Rhtr+Rhcorr);
        iterr = Ttpl+Taux-Thtr;
        if (fabs(iterr)<0.01) break;
        if (fabs(iterr)>10000.)
        {
           Rhcorr=0.;
           break;
        }
        Rhcorr += iterr*0.1;
      }
    }
```